In [1]:
import boto3
import pandas as pd
import io, logging

In [2]:
session = boto3.Session(profile_name='aws')

In [3]:
s3_res = session.resource('s3')
s3_client = session.client('s3')

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/erkansirin78/datasets/master/Advertising.csv")

In [5]:
df.head()

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


# Save df to s3 bucket

In [6]:
def save_df_to_s3(df, bucket, key):
    ''' Store df as a buffer, then save buffer to s3'''
    try:
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        # s3_resource = boto3.resource('s3')
        s3_res.Object(bucket, key).put(Body=csv_buffer.getvalue())
        logging.info(f'{key} saved to s3 bucket {bucket}')
    except Exception as e:
        raise logging.exception(e)

In [7]:
save_df_to_s3(df=df, bucket='vbo-de-bootcamp', key='mlops2/Advertising.csv')

# Read df from s3 bucket

In [8]:
def load_df_from_s3(bucket, key, s3_client, index_col=None, usecols=None, sep=","):
    ''' Read a csv from a s3 bucket & load into pandas dataframe'''
    try:
        logging.info(f"Loading {bucket, key}")
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        return pd.read_csv(obj['Body'], index_col=index_col, usecols=usecols, low_memory=False, sep=sep)
    except Exception as e:
        raise logging.exception(e)

In [10]:
df_from_s3 = load_df_from_s3(bucket='vbo-de-bootcamp', key='mlops2/Advertising.csv', s3_client=s3_client)

In [11]:
df_from_s3.head()

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
